In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import normalize
# from sklearn.cluster import KMeans, DBSCAN
# import seaborn as sns
# import matplotlib.pyplot as plt
# import math

In [5]:
from dėstytojo import LKS94WGS84
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

# Suskirstykite eismo įvykius į klasterius (2022 metų) 
# (pagal koordinantes, atskirai kiekvienai eismo įvykio rūšiai, su DBSCAN bei KMeans).
# Darydami KMeans klasterizaciją, pasinaudokite elbow grafiku optimaliam klasterių skaičiaus nustatymui (Jei pavyksta)

# Patekite vizualizacijas klasterių išsidėstymo su mapbox()
# Pasirinkite vieną eismo įvykių rūšį, ir su GridsearchCV, GridSearch klase suraskite optimalų klasterių kiekį

# Tada taip pat pateikite klasterių vizualizacijas.

In [3]:
df = pd.read_json(path_or_buf='https://data.gov.lt/media/filer_public/5e/b8/5eb89d55-f57a-48ec-b1a2-063692c1b6a7/ei_2022_12_31.json', convert_dates=['dataLaikas'])
df = df[['dataLaikas', 'iskaitinis','dalyviuSkaicius','rusis', 'ilguma','platuma']].dropna(subset=['ilguma','platuma'])

In [7]:
LAT, LON = [],[]

for ilguma, platuma in zip(df['ilguma'],df['platuma']):
    lat, lon = LKS94WGS84.grid2geo(platuma, ilguma)
    LAT.append(lat)
    LON.append(lon)

df['long'] = LON
df['lat'] = LAT

In [9]:
df.head(3)

,dataLaikas,iskaitinis,dalyviuSkaicius,rusis,ilguma,platuma,long,lat
0,2021-12-31 22:00:00,0,1.0,Kiti eismo įvykiai,6083759.0,491606.0,23.869164,54.889850
1,2022-01-01 00:40:00,0,2.0,Susidūrimas,6181902.0,317439.0,21.092653,55.737249
2,2022-01-01 02:02:00,0,1.0,Užvažiavimas ant gyvūno,6161589.0,333263.0,21.356655,55.560661


In [10]:
tp_df = df.groupby('rusis').size().sort_values(ascending=False).reset_index(name='count')
tp_df['count_diff'] = ((tp_df['count'].shift(1)-tp_df['count'])/tp_df['count']).fillna(0)
print(tp_df)

                                           rusis  count  count_diff
0                                    Susidūrimas   5319    0.000000
1                        Užvažiavimas ant gyvūno   4927    0.079562
2   Susidūrimas su stovinčia transporto priemone   4799    0.026672
3                             Kiti eismo įvykiai   4235    0.133176
4                      Užvažiavimas ant kliūties   1949    1.172909
5                      Užvažiavimas ant pėsčiojo    843    1.311981
6                                     Apvirtimas    663    0.271493
7                        Susidūrimas su dviračiu    561    0.181818
8                       Susidūrimas su motociklu    121    3.636364
9                          Susidūrimas su mopedu     62    0.951613
10     Susidūrimas su bėgine transporto priemone      4   14.500000


In [11]:
for i in range(0,8):
    print(f"Kiekis: {tp_df.iloc[i]['count']}, tipas: {tp_df.iloc[i]['rusis']}")

# print(df.loc[i,'x'])
# print(df.iloc[i]['x'])


Kiekis: 5319, tipas: Susidūrimas
Kiekis: 4927, tipas: Užvažiavimas ant gyvūno
Kiekis: 4799, tipas: Susidūrimas su stovinčia transporto priemone
Kiekis: 4235, tipas: Kiti eismo įvykiai
Kiekis: 1949, tipas: Užvažiavimas ant kliūties
Kiekis: 843, tipas: Užvažiavimas ant pėsčiojo
Kiekis: 663, tipas: Apvirtimas
Kiekis: 561, tipas: Susidūrimas su dviračiu


In [12]:
import plotly.express as px

# df.query('rusis == "Užvažiavimas ant gyvūno"', inplace=True)

for i in range(0,8):
    print(f"{tp_df.iloc[i]['rusis']} (n={tp_df.iloc[i]['count']})")
    df_ = df[df['rusis']==tp_df.iloc[i]['rusis']]
    # print(len(df),len(df_))
    
    fig = px.scatter_mapbox(data_frame=df_, lat='lat', lon='long', zoom=6)
    fig.update_layout(mapbox_style="open-street-map") # open-street-map, carto-darkmatter
    fig.update_layout(width=800, height=600)
    fig.show()
    


Susidūrimas (n=5319)


Užvažiavimas ant gyvūno (n=4927)


Susidūrimas su stovinčia transporto priemone (n=4799)


Kiti eismo įvykiai (n=4235)


Užvažiavimas ant kliūties (n=1949)


Užvažiavimas ant pėsčiojo (n=843)


Apvirtimas (n=663)


Susidūrimas su dviračiu (n=561)
